In [ ]:
# Step 1: Import Libraries

from pathlib import Path
from piecewise import *
import pickle

In [ ]:
# Step 2: Define data paths

parent_folder = os.path.join(*Path(os.getcwd()).parts[:-2], "LV Strain Curves")
data_path = os.path.join(parent_folder, "Data/EPOGH/Strain curves (Filtered)")
save_data_path = os.path.join(parent_folder, "Results/Summary Index/GMM/EPOGH/")

# create save_data_path directory if not exist
if not os.path.exists(save_data_path):
    os.makedirs(save_data_path)

avc_files = ["Data/EPOGH/EPOGH_AVC.xlsx"]

In [ ]:
# Step 3: Load the data

original_data, data, patient_id, interval = ld.read_data(data_path)
excluded_patients1, avc_times = ld.read_avc_time(parent_folder, avc_files)
excluded_patients2 = []
p_wave_times = pd.DataFrame(columns=["ID", "PQ Interval", "P duration (ms)"])

original_data, data, patient_id, interval = exclude_patients(excluded_patients1, excluded_patients2, original_data,
                                                             data, patient_id, interval)
decision = "peak"
reference_patient_id = "110101"
_, deformation, _, _, reference_time = get_aligned_signals(original_data, decision, interval, patient_id,
                                                           reference_patient_id, avc_times, p_wave_times)

deformation = np.array(deformation)
patient_id = np.array(patient_id)

In [ ]:
# Step 4a: Extract the desired features

extracted_features = extract_time_series_features(reference_time, deformation, patient_id, save_data_path,
                                                  do_plot=False)

training_data = {"peak": extracted_features["Peak"],
                 "peak_slopes": extracted_features[["Systolic Slope", "Diastolic Slope","Peak"]],
                 "all":extracted_features}

In [ ]:
# Step 4b: Select the desired feature-set

select_features = "all"
save_data_path = os.path.join(save_data_path, select_features)
if not os.path.exists(save_data_path):
    os.makedirs(save_data_path)

In [ ]:
# Step 5: Load the trained Gaussian Mixture Model

clustering_model = pickle.load(open(os.path.join(save_data_path,"gmm_model.pkl"), 'rb'))
clusters = clustering_model.predict(training_data[select_features])
representative_centers = np.array(produce_centroids(clusters, deformation))

In [ ]:
# Step 6: Plot the results in EPOGH cohort

cluster_labels = {0:3, 1:4, 2:2, 3:1}
cluster_colours = {0:"blueviolet", 1:"red", 2:"blue", 3:"green", 4:"blueviolet"}
patient_id = patient_id.tolist()
# groups the patients' ids per cluster. Returns a list
clustered_id = analyze_patient(clusters, patient_id)

# writes an excel file with the patient's ID and its assigned cluster label
write2excel(clusters, patient_id, save_data_path, cluster_labels)

# plots the strain traces grouped per cluster with Matplotlib and plotly. Matplotlib produces png and svg files.
visualize_clustering_results(reference_time, deformation, clustered_id, clusters, patient_id, representative_centers,
                             save_data_path, cluster_labels=cluster_labels, cluster_colours=cluster_colours)

# plot the first three principal components of the strain curves
plot_pca(clusters, deformation, patient_id, save_data_path, cluster_labels=cluster_labels,
         cluster_colours=cluster_colours)

# plots the gradient of the Strain traces with Matplotlib and plotly.
plot_gradients(deformation, reference_time, clusters, clustered_id,patient_id, save_data_path,
               cluster_labels=cluster_labels, cluster_colours=cluster_colours)